# Sincronização com Google Drive

In [6]:
import os, string, time
from google.colab import drive
drive.mount('/content/drive')
from google.colab import output

def atualizarConsole(nova_string : str) -> None:
  output.clear()
  print(nova_string)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# Instalação e importação de bibliotecas que serão utilizadas

In [7]:
string_exibida = 'Importando string, os, re...\n'
atualizarConsole(string_exibida)
import string
import os
import re
# import msgpack
string_exibida = '''Importando string: OK
Importando OS: OK
Importando re: OK
Instalando e importando pt-br-verbs-lemmatizer...\n'''
atualizarConsole(string_exibida)
try:
  !pip install pt-br-verbs-lemmatizer -U
  from pt_br_verbs_lemmatizer import lemmatize as lematizar, modifyDicAmbiguousVerbs as modificarAmbiguidadeVerbal
except Exception as e:
  print(f'Problema na instalação e importação de pt-br-verbs-lemmatizer:\n{e.__class__.__name__}: {str(e)}')
else:
  string_exibida = '''Importando string: OK
Importando OS: OK
Importando re: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln...\n'''
  atualizarConsole(string_exibida)
  try:
    !pip install ferramentas-basicas-pln -U
    from ferramentas_basicas_pln import formatarTexto,removerCaracteresEspeciais,removerCaracteresEstranhos,removerEspacosEmBrancoExtras,transformarTextoSubstituindoCaracteres,coletarTextoDeArquivoTxt
    from ferramentas_basicas_pln import STRING_CARACTERES_ESPECIAIS_PADRAO
  except Exception as e:
    print(f'Problema na instalação e importação de ferramentas:\n{e.__class__.__name__}: {str(e)}')
  else:
    string_exibida = '''Importando string: OK
Importando OS: OK
Importando re: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos...\n'''
    atualizarConsole(string_exibida)
    try:
      import nltk
      from nltk.tokenize import sent_tokenize, word_tokenize
      nltk.download('punkt')
      nltk.download('stopwords')
    except Exception as e:
      print(f'Problema na instalação e importação ou download de módulos de NLTK:\n{e.__class__.__name__}: {str(e)}')
    else:
      string_exibida = '''Importando string: OK
Importando OS: OK
Importando re: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos: OK
Instalando e importando spaCy e fazendo download de módulos...\n'''
      atualizarConsole(string_exibida)
      try:
        !pip install -U spacy
        !pip install -U spacy-lookups-data
        !python -m spacy download pt_core_news_lg

        import spacy
        # nlp_spacy = spacy.load('pt_core_news_lg')
        nlp_spacy = spacy.load('pt_core_news_lg',enable=["tok2vec","ner","morphologizer"])
        # nlp_spacy.enable_pipe("senter")
      except Exception as e:
        print(f'Problema na instalação e importação ou download de módulos de spaCy:\n{e.__class__.__name__}: {str(e)}')
      else:
        string_exibida = '''Importando string: OK
Importando OS: OK
Importando re: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos: OK
Instalando e importando spaCy e fazendo download de módulos: OK
'''+'\n'+'-'*100+'\n'+'Ambiente configurado com sucesso.'+'\n'+'-'*100+'\n'
        atualizarConsole(string_exibida)

Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos: OK
Instalando e importando spaCy e fazendo download de módulos: OK

----------------------------------------------------------------------------------------------------
Ambiente configurado com sucesso.
----------------------------------------------------------------------------------------------------



# Lista de entidades nomeadas personalizada

In [8]:
string_entidades_manuais = '''História cultural, história global, história social, história social da cultura, histórias entrelaçadas, histórias entrecruzadas, história conectada, história transnacional, filosofia da história, teoria da história, teoria e filosofia da história, teoria marxista, teoria queer, sistema mundo, racismo estrutural, história da historiografia, teoria do sistema-mundo, história ambiental, história das mulheres, história indígena, história global do trabalho, história do trabalho, história oral, estudos de gênero, História militar, história das religiões, história da arte, história econômica, Guerra do Contestado, Movimento Social do Contestado, Teoria Racial, Historiografia Literária'''

string_entidades_manuais = removerCaracteresEstranhos(string_entidades_manuais)#.replace('\xa0',' ').replace('\n','')

lista_entidades_nomeadas_manuais = [elemento.lower().strip() for elemento in string_entidades_manuais.split(',')]

lista_entidades_nomeadas_manuais_ordenada = sorted(lista_entidades_nomeadas_manuais, key=len, reverse=True)

lista_entidades_nomeadas_manuais_ordenada

['teoria e filosofia da história',
 'movimento social do contestado',
 'história global do trabalho',
 'história social da cultura',
 'história da historiografia',
 'historiografia literária',
 'histórias entrecruzadas',
 'teoria do sistema-mundo',
 'histórias entrelaçadas',
 'história transnacional',
 'história das religiões',
 'filosofia da história',
 'história das mulheres',
 'história do trabalho',
 'guerra do contestado',
 'história conectada',
 'teoria da história',
 'racismo estrutural',
 'história ambiental',
 'história econômica',
 'história cultural',
 'história indígena',
 'estudos de gênero',
 'história militar',
 'história da arte',
 'história global',
 'história social',
 'teoria marxista',
 'sistema mundo',
 'história oral',
 'teoria racial',
 'teoria queer']

# Lista de stopwords a ser utilizada

In [9]:
stopwords = '''#Artigos
o
os
a
as
uns
umas
#Preposições
à
às
aos
da
das
do
dos
no
nos
na
nas
numa
numas
num
nuns
dessa
dessas
desse
desses
desta
destas
deste
destes
ante
até
após
com
contra
de
desde
em
entre
para
perante
por
sem
sob
sobre
trás
conforme
consoante
mediante
tirante
senão
#Pronomes
eu
tu
ele
eles
ela
elas
nós
vós
me
te
lhe
lhes
se
nos
vos
mim
contigo
ti
contigo
si
consigo
conosco
convosco
#Conjunções
mas
e
também
só
todavia
ou
-seja
portanto
logo
porque
que
assim
já
embora
ainda
conforme
depois
antes
afim
quanto
quantos
quanta
quantas
#Extras
pela
pelas
pelo
pelos
qual
quais
quando
quem
aquele
àquele
aqueles
àqueles
aquela
àquela
aquelas
àquelas
aquilo
àquilo
naquele
naqueles
naquela
naquelas
naquilo'''

lista_stopwords = [palavra.strip() for palavra in stopwords.split('\n') if not (palavra.startswith('#') or palavra.startswith('-'))]
print('Lista de stopwords construída sem filtro de repetição, tamanho:',len(lista_stopwords))
print('-'*100)
print('Palavras repetidas encontradas:')
for p in set([elemento for elemento in lista_stopwords if lista_stopwords.count(elemento) > 1]):
  print(p)
print('-'*100)
print('Aplicando filtro...')
print('-'*100)
lista_stopwords = sorted(set([palavra.strip() for palavra in stopwords.split('\n') if not (palavra.startswith('#') or palavra.startswith('-'))]),key=len)
print('Lista de stopwords construída com filtro de repetição, tamanho atualizado:',len(lista_stopwords))

Lista de stopwords construída sem filtro de repetição, tamanho: 118
----------------------------------------------------------------------------------------------------
Palavras repetidas encontradas:
conforme
nos
contigo
----------------------------------------------------------------------------------------------------
Aplicando filtro...
----------------------------------------------------------------------------------------------------
Lista de stopwords construída com filtro de repetição, tamanho atualizado: 115


# Modificação da lematização baseada em regras (removendo ambiguidades/duplicidades)

In [11]:
print('Modificando dataset ambiguidades verbais na lematização\nSeleção Mateus...')
modificarAmbiguidadeVerbal(type='infinitive',verbs=['impostar','incender','interditar',
                                                    'interver','irar','iriar','fossar',
                                                    'liar','mantar','mentar','mentirar',
                                                    'podar','possar','preditar',
                                                    'postar','querar','querendar','raer',
                                                    'recobrar','redizer','refugir','remedar',
                                                    'repelar','revir','rer','ruar','segar',
                                                    'erar','seriar','suster'])

modificarAmbiguidadeVerbal(type='flex',verbs=[('relemos','relar'),('relemo-lo','relar'),
                                              ('retemos','retar'),('retemo-lo','retar'),('retemo-nos','retar'),
                                              ('revira','rever'),('reviram','rever'),('reviras','rever'),('revirem','rever'),('revires','rever'),('revira-o','rever'),('revira-lo','rever'),
                                              ('reviste','rever'),('revisto','rever'),('revistes','rever'),('reviste-o','rever'),('revisto-a','rever'),('revisto-o','rever'),('reviste-te','rever'),('revisto-as','rever'),('revisto-me','rever'),('revisto-os','rever'),('revisto-se','rever'),('revisto-te','rever'),
                                              ('revisto-o','rever'),('revisto-me','rever'),
                                              ('ruido-a','ruidar'),('ruido-o','ruidar'),('ruido-as','ruidar'),('ruido-me','ruidar'),('ruido-os','ruidar'),('ruido-se','ruidar'),('ruido-te','ruidar'),
                                              ('sede','ser'),('sede-o','ser'),
                                              ('surta','sortir'),('surte','sortir'),('surto','sortir'),('surtam','sortir'),('surtas','sortir'),('surtem','sortir'),('surtes','sortir'),('surta-o','sortir'),('surtais','sortir'),('surte-o','sortir'),('surto-o','sortir'),('surta-se','sortir'),('surtamos','sortir'),('surte-se','sortir'),('surte-te','sortir'),('surto-me','sortir'),('surtam-no','sortir'),('surtam-se','sortir'),('surtem-no','sortir'),('surtem-se','sortir'),('surtamo-nos','sortir'),
                                              ('some-o','sumir'),('some-se','sumir'),('some-te','sumir'),('somem-no','sumir'),('somem-se','sumir')])
print('Feito')

print('Modificando dataset ambiguidades verbais na lematização\nSeleção Carlos...')
modificarAmbiguidadeVerbal(type='infinitive',verbs=['valar','vestar','vinhar','abolar',
                                                    'açalmar','alardar','amandar','amolecar',
                                                    'apresar','arrefeçar','beberar','bombardar'])

modificarAmbiguidadeVerbal(type='flex',verbs=[('traga','trager'),('trago','trager'),('tragam','trager'),('tragas','trager'),('traga-o','trager'),('tragais','trager'),('trago-o','trager'),('tragamos','trager'),('tragam-no','trager'),
                                              ('abraque','abraçar'),('abracado','abraçar'),('abracará','abraçar'),('abrace-a','abraçar'),('abrace-o','abraçar'),('abraceis','abraçar'),('abraquei','abraçar'),('abraquem','abraçar'),('abraques','abraçar'),('abracai-a','abraçar'),('abracai-o','abraçar'),('abracarás','abraçar'),('abracarão','abraçar'),
                                              ('abracente','abraçar'),('abraque-o','abraçar'),('abraqueis','abraçar'),('abracado-a','abraçar'),('abracado-o','abraçar'),('abracai-as','abraçar'),('abracai-os','abraçar'),
                                              ('abracáveis','abraçar'),('abraquei-o','abraçar'),('abraquemos','abraçar'),('abracado-as','abraçar'),('abracado-la','abraçar'),('abracado-lo','abraçar'),('abracado-me','abraçar'),('abracado-na','abraçar'),('abracado-no','abraçar'),('abracado-os','abraçar'),('abracado-se','abraçar'),('abracado-te','abraçar'),('abracai-lhe','abraçar'),('abracai-vos','abraçar'),('abracaríeis','abraçar'),('abracei-lhe','abraçar'),('abracem-lhe','abraçar'),('abracem-nas','abraçar'),('abracem-nos','abraçar'),('abracemô-la','abraçar'),('abracemô-lo','abraçar'),('abracá-lo-á','abraçar'),('abracáramos','abraçar'),('abracásseis','abraçar'),('abracávamos','abraçar'),('abraquem-no','abraçar'),('abracado-las','abraçar'),('abracado-lhe','abraçar'),('abracado-los','abraçar'),('abracado-nas','abraçar'),('abracado-nos','abraçar'),('abracai-lhes','abraçar'),('abracando-la','abraçar'),('abracando-lo','abraçar'),('abracaríamos','abraçar'),
                                              ('abracá-lo-ei','abraçar'),('abracá-lo-ia','abraçar'),('abracá-lo-ás','abraçar'),('abracá-lo-ão','abraçar'),('abracárei-lo','abraçar'),('abracássemos','abraçar'),('abracávei-lo','abraçar'),('abraquemo-lo','abraçar'),('abracado-lhes','abraçar'),('abracando-las','abraçar'),('abracando-los','abraçar'),('abracemo-lhes','abraçar'),('abracá-lo-eis','abraçar'),('abracá-lo-iam','abraçar'),('abracá-lo-ias','abraçar'),('abracáramo-lo','abraçar'),('abracávamo-lo','abraçar'),('abracá-lo-emos','abraçar'),('abracá-lo-íeis','abraçar'),('abracá-lo-íamos','abraçar'),
                                              ('acaba-a','acaber'),('acaba-o','acaber'),('acabais','acaber'),('acabe-a','acaber'),('acabe-o','acaber'),('acabeis','acaber'),('acabo-a','acaber'),('acabo-o','acaber'),('acaba-as','acaber'),('acaba-me','acaber'),('acaba-os','acaber'),('acaba-se','acaber'),('acaba-te','acaber'),
                                              ('acabe-me','acaber'),('acabe-os','acaber'),('acabe-se','acaber'),('acabe-te','acaber'),('acabei-a','acaber'),('acabei-o','acaber'),
                                              ('acabo-as','acaber'),('acabo-me','acaber'),('acabo-os','acaber'),('acabo-se','acaber'),('acabo-te','acaber'),('acabam-me','acaber'),('acabam-na','acaber'),('acabam-no','acaber'),('acabam-se','acaber'),('acabam-te','acaber'),('acabei-as','acaber'),('acabei-os','acaber'),('acabem-me','acaber'),('acabem-na','acaber'),('acabem-no','acaber'),('acabem-se','acaber'),('acabem-te','acaber'),('acabam-lhe','acaber'),('acabam-nas','acaber'),('acabam-nos','acaber'),('acabamô-la','acaber'),('acabamô-lo','acaber'),('acabei-lhe','acaber'),('acabem-lhe','acaber'),('acabem-nas','acaber'),('acabem-nos','acaber'),('acabemô-la','acaber'),('acabemô-lo','acaber'),('acabam-lhes','acaber'),('acabamo-lhe','acaber'),('acabamo-nas','acaber'),('acabamo-nos','acaber'),('acabamô-las','acaber'),('acabamô-los','acaber'),('acabei-lhes','acaber'),('acabem-lhes','acaber'),('acabemo-lhe','acaber'),('acabemo-nas','acaber'),('acabemo-nos','acaber'),('acabemô-las','acaber'),('acabemô-los','acaber'),('acabamo-lhes','acaber'),('acabemo-lhes','acaber'),
                                              ('achara-a','acharar'),('achara-o','acharar'),('achara-as','acharar'),('achara-os','acharar'),
                                              ('acoitado-me','açoitar'),('acoitáveis','açoitar'),
                                              ('adia','adir'),('adiam','adir'),('adias','adir'),('adia-a','adir'),('adia-o','adir'),('adia-as','adir'),('adia-me','adir'),('adia-os','adir'),('adia-se','adir'),('adia-te','adir'),('adiá-la','adir'),('adiá-lo','adir'),('adia-lhe','adir'),('adiam-me','adir'),('adiam-na','adir'),('adiam-no','adir'),('adiam-se','adir'),('adiam-te','adir'),('adias-me','adir'),('adias-se','adir'),('adias-te','adir'),('adiá-las','adir'),('adiá-los','adir'),('adia-lhes','adir'),('adiam-lhe','adir'),('adiam-nas','adir'),('adiam-nos','adir'),('adiam-lhes','adir'),
                                              ('amente','amar'),
                                              ('assente','assar'),('atente','atar'),
                                              ('atendam-se','atendar'),('atendam-te','atendar'),('atendei-as','atendar'),('atendei-os','atendar'),('atendem-se','atendar'),('atendem-te','atendar'),('atendam-lhe','atendar'),('atendam-nas','atendar'),('atendam-nos','atendar'),('atendamô-la','atendar'),('atendamô-lo','atendar'),('atendei-lhe','atendar'),('atendem-lhe','atendar'),('atendem-nas','atendar'),('atendem-nos','atendar'),('atendemô-la','atendar'),('atendemô-lo','atendar'),('atendam-lhes','atendar'),('atendamo-lhe','atendar'),('atendamo-nas','atendar'),('atendamo-nos','atendar'),('atendamô-las','atendar'),('atendamô-los','atendar'),('atendei-lhes','atendar'),('atendem-lhes','atendar'),('atendemo-lhe','atendar'),('atendemo-nas','atendar'),('atendemo-nos','atendar'),('atendemô-las','atendar'),('atendemô-los','atendar'),('atendamo-lhes','atendar'),('atendemo-lhes','atendar'),
                                              ('avivente','avivar'),
                                              ('calque','calçar'),('calquei','calçar'),('calquem','calçar'),('calques','calçar'),('calquei-o','calçar'),('calquemos','calçar')])
print('Feito')

Modificando dataset ambiguidades verbais na lematização
Seleção Mateus...
Feito
Modificando dataset ambiguidades verbais na lematização
Seleção Carlos...
Feito


# Funções

In [25]:
def padronizacaoTexto(texto : str) -> str:
  texto = texto.replace('\n',' ')
  return formatarTexto(texto=texto,
                       padronizar_texto_para_minuscula=False,
                       remover_caracteres_especiais=False,
                       remover_caracteres_estranhos=True,
                       remover_espacos_em_branco_em_excesso=True,
                       padronizar_links=True,padrao_link='link_url_mask',
                       padronizar_dinheiros=True,padrao_dinheiro='R$',
                       padronizar_emails=True,padrao_email='e_mail_mask')

def padronizacaoTokens(lista_frases_tokenizadas : list[str],
                       lista_tokens_personalizados : list[str] = []) -> str:
  lista_frases_tokenizadas_padronizada = []
  lista_tokens_personalizados += ['link_url_mask','R$','e_mail_mask']

  for frase in lista_frases_tokenizadas:
    lista_frases_tokenizadas_padronizada_atual = []
    for token in frase:
      # if len(token.split()) > 1: # spaCy lematizando no = 'em o'
      #   for t in token.split():
      #     if (t not in STRING_CARACTERES_ESPECIAIS_PADRAO) and (t.lower() not in lista_stopwords):
      #       if t not in lista_tokens_personalizados:
      #         t = formatarTexto(texto=t,
      #                           remover_caracteres_especiais=True,
      #                           remover_caracteres_estranhos=False,
      #                           remover_espacos_em_branco_em_excesso=False,
      #                           padronizar_texto_para_minuscula=True).strip()
      #       else:
      #         t = t.strip()

      #       if t:
      #         lista_frases_tokenizadas_padronizada_atual.append(t)
      # else:
      if (token not in STRING_CARACTERES_ESPECIAIS_PADRAO) and (token.lower() not in lista_stopwords):
        if token not in lista_tokens_personalizados:
          token = formatarTexto(texto=token,
                                remover_caracteres_especiais=True,
                                remover_caracteres_estranhos=False,
                                remover_espacos_em_branco_em_excesso=False,
                                padronizar_texto_para_minuscula=True).strip()
        else:
          token = token.strip()

        if token:
          lista_frases_tokenizadas_padronizada_atual.append(token)


    if lista_frases_tokenizadas_padronizada_atual:
      lista_frases_tokenizadas_padronizada.append(lista_frases_tokenizadas_padronizada_atual)

  if lista_frases_tokenizadas_padronizada:
    return lista_frases_tokenizadas_padronizada

def substituirEntidadeNomeadaManual(texto : str,
                                    entidade_normal : str,
                                    entidade_formatada : str) -> str:
  return re.sub(r'{x}'.format(x=entidade_normal), entidade_formatada, texto, flags=re.IGNORECASE)


def salvarArquivoMsgPack(nome_variavel : str,
                         variavel_em_questao,
                         pasta_para_salvar : str = '') -> tuple[bool,str]:
    try:
        if not os.path.exists(pasta_para_salvar):
            os.makedirs(pasta_para_salvar)

        variable_bytes = msgpack.packb(variavel_em_questao)

        if not nome_variavel.endswith('.msgpack'):
            nome_variavel += '.msgpack'

        with open(os.path.join(pasta_para_salvar,nome_variavel),'wb') as f:
            f.write(variable_bytes)
            f.close()
            return True, f'Variável {nome_variavel} salva com sucesso no formato .msgpack'

    except Exception as e:
        error_message = f'{e.__class__.__name__}: {str(e)}'
        return False, error_message

def preProcessarTextoComRE(texto : str,
                           caminho_para_salvar_tokenizacao : str) -> list:

  # texto = coletarTextoDeArquivoTxt(caminho_arquivo=caminho_arquivo_texto,tipo_de_encoding='utf-8')

  if texto:

    texto = padronizacaoTexto(texto)

    lista_entidades = []
    lista_frases_tokenizadas = []

    sentencas = sent_tokenize(texto, language='portuguese')

    for sentenca in sentencas:
      doc = nlp_spacy(sentenca)

      lista_tuplas_entidades = [(ent.orth_,transformarTextoSubstituindoCaracteres(texto=formatarTexto(texto=ent.orth_,
                                                                                                      remover_caracteres_estranhos=False,
                                                                                                      remover_espacos_em_branco_em_excesso=False,),
                                                                                  caracteres=' ',
                                                                                  substituir_por='_')) for ent in doc.ents]

      lista_tuplas_entidades_manuais = [(entidade_normal, transformarTextoSubstituindoCaracteres(texto=formatarTexto(texto=entidade_normal,
                                                                                                        remover_caracteres_estranhos=False,
                                                                                                        remover_espacos_em_branco_em_excesso=False),
                                                                                                  caracteres=' ',
                                                                                                  substituir_por='_')) for entidade_normal in lista_entidades_nomeadas_manuais_ordenada]
      lista_atual = []

      for entidade_normal,entidade_formatada in lista_tuplas_entidades:
        sentenca = sentenca.replace(entidade_normal,entidade_formatada)
        lista_entidades.append(entidade_formatada)

      for entidade_normal, entidade_formatada in lista_tuplas_entidades_manuais:
        sentenca = substituirEntidadeNomeadaManual(texto=sentenca,entidade_normal=entidade_normal,entidade_formatada=entidade_formatada)
        lista_entidades.append(entidade_formatada)

      for palavra in doc:
        try:
          if (not palavra.is_punct):
            if palavra.orth_ not in lista_entidades:
              if palavra.pos_ in ['VERB','AUX']:
                lista_atual.append(lematizar(verb=palavra.orth_,ignore_ambiguous_verbs=False))
              else:
                lista_atual.append(palavra.orth_)
            else:
              lista_atual.append(palavra.orth_)
        except Exception as e:
          erro = f'{e.__class__.__name__}: {str(e)}'
          # print(f'Erro ao processar texto na frase "{sentenca}":\n\t-->{erro}.\nProcesso de Tokenização com lematização total / Sem rec. de ents / spaCy')
          pass

      if lista_atual:
        lista_frases_tokenizadas.append(lista_atual)

    if lista_frases_tokenizadas:
      lista_tokens_personalizados = [entidade for entidade in lista_entidades]
      lista_frases_tokenizadas = padronizacaoTokens(lista_frases_tokenizadas=lista_frases_tokenizadas,lista_tokens_personalizados=lista_tokens_personalizados)

      if lista_frases_tokenizadas:
        status_save, msg_save = salvarArquivoMsgPack(nome_variavel='pre_processamento_c_re',
                                                     variavel_em_questao=lista_frases_tokenizadas,
                                                     pasta_para_salvar=caminho_para_salvar_tokenizacao)
        if not status_save:
          print('\n!',msg_save,'\n\n')
          return False
        else:
          return True

  return False

# Execução

In [ ]:
caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções_Teste'

caminho_colecoes_word_embeddings = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Resultados_Tokenizacao/Colecoes_Teste'

lista_de_colecoes_extracao = [os.path.join(caminho_colecoes_extracao_textos,colecao) for colecao in os.listdir(caminho_colecoes_extracao_textos) if '.' not in colecao]

for colecao in lista_de_colecoes_extracao:
  lista_de_anos_colecao_extracao = [os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()]
  for ano in lista_de_anos_colecao_extracao:
    lista_trabalhos_ano_colecao_extracao = [os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')]
    for trabalho in lista_trabalhos_ano_colecao_extracao:
      lista_arquivos_trabalho_ano_colecao_extracao = [os.path.join(trabalho,arquivo) for arquivo in os.listdir(trabalho) if arquivo in ['notas_de_rodape.txt','texto_principal.txt']]
      txt_total = ''
      for arquivo in lista_arquivos_trabalho_ano_colecao_extracao:
        try:
          txt_total += coletarTextoDeArquivoTxt(caminho_arquivo=arquivo,tipo_de_encoding='utf-8')+' '
        except Exception as e:
          pass
      if txt_total != '':
        caminho_pasta_salvar_pre_processamento = os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho))
        if not os.path.exists(caminho_pasta_salvar_pre_processamento):
          os.makedirs(caminho_pasta_salvar_pre_processamento)
        if preProcessarTextoComRE(texto=txt_total,caminho_para_salvar_tokenizacao=caminho_pasta_salvar_pre_processamento):
          pass
        else:
          print('\n! Problema ao salvar arquivo de pre_processamento de',colecao,ano,trabalho,'\n')

# Teste de abertura dos arquivos armazenados

In [30]:
def openMsgPackFile(full_filepath : str,
                    encoding_type : str = None):
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded

for colecao in os.listdir(caminho_colecoes_word_embeddings):
  for ano in os.listdir(os.path.join(caminho_colecoes_word_embeddings,colecao)):
    for trabalho in os.listdir(os.path.join(caminho_colecoes_word_embeddings,colecao,ano)):
      for arquivo in os.listdir(os.path.join(caminho_colecoes_word_embeddings,colecao,ano,trabalho)):
        print(colecao,ano,trabalho,arquivo)
        frases_tokenizadas = openMsgPackFile(full_filepath=os.path.join(caminho_colecoes_word_embeddings,colecao,ano,trabalho,arquivo))
        print(frases_tokenizadas)
        print('--- frases separadas')
        for frase in frases_tokenizadas:
          print(frase)
        print('\n'+'-'*100+'\n')
      print('')
    print('')
  print('')
